<a href="https://colab.research.google.com/github/wafaabek/Big_data/blob/main/Spark_GraphX_et_GraphFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Chargement des fichiers en DataFrames
# Tout d'abord, chargeons les deux fichiers station_data.csv et trip_data.csv en DataFrames.

In [1]:
from pyspark.sql import SparkSession

# Créer une session Spark
spark = SparkSession.builder \
    .appName("TP5 Spark GraphX et GraphFrames") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.1-spark3.0-s_2.12") \
    .getOrCreate()

# Charger les fichiers CSV en DataFrames
station_data_df = spark.read.csv("/content/sample_data/station_data.csv", header=True, inferSchema=True)
trip_data_df = spark.read.csv("/content/sample_data/trip_data.csv", header=True, inferSchema=True)


# 2. Renommer les colonnes pour les sommets et les arcs
# Suivez les conventions de GraphFrames pour renommer les colonnes de façon appropriée.

In [3]:
# Renommer la colonne 'name' en 'id' dans station_data_df
station_vertices = station_data_df.withColumnRenamed("name", "id")

# Renommer les colonnes 'Start Station' en 'src' et 'End Station' en 'dst' dans trip_data_df
trip_edges = trip_data_df.withColumnRenamed("Start Station", "src") \
                         .withColumnRenamed("End Station", "dst")


# 3. Créer un GraphFrame
# Utilisez le DataFrame des sommets (station_vertices) et des arcs (trip_edges) pour construire un graphe.

In [4]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pyspark.sql.dataframe")

from graphframes import GraphFrame

# Renommer la colonne 'name' en 'id' pour les sommets (stations)
station_vertices = station_data_df.withColumnRenamed("name", "id")

# Créer le GraphFrame
graph = GraphFrame(station_vertices, trip_edges)



# Interroger le graphe

5. Nombre de voyages entre chaque source et destination, triés par ordre décroissant

In [15]:
from pyspark.sql.functions import col

# Compter le nombre de trajets entre chaque source et destination
trip_counts = graph.edges.groupBy("src", "dst").count().orderBy(col("count").desc())

# Afficher les résultats
trip_counts.show()

+--------------------+--------------------+-----+
|                 src|                 dst|count|
+--------------------+--------------------+-----+
|San Francisco Cal...|     Townsend at 7th|    4|
|       5th at Howard|San Francisco Cal...|    3|
|San Francisco Cal...|  Powell Street BART|    2|
|     2nd at Townsend|   Market at Sansome|    2|
|     Spear at Folsom|     2nd at Townsend|    2|
|   Market at Sansome|Broadway St at Ba...|    2|
|    Davis at Jackson|Embarcadero at Sa...|    2|
|San Francisco Cal...|   2nd at South Park|    2|
|   Steuart at Market|San Francisco Cal...|    2|
|       Market at 4th|San Francisco Cal...|    2|
|Embarcadero at Fo...|Embarcadero at Sa...|    2|
|Mountain View Cal...|Rengstorff Avenue...|    1|
|    Davis at Jackson|Temporary Transba...|    1|
|       5th at Howard|     Townsend at 7th|    1|
|   Market at Sansome|South Van Ness at...|    1|
|     Beale at Market|Temporary Transba...|    1|
|     2nd at Townsend|Powell at Post (U...|    1|


6. Nombre de voyages se terminant ou commençant à ‘Townsend at 7th’

In [18]:
from pyspark.sql.functions import col

# Filtrer les trajets qui commencent ou se terminent à "Townsend at 7th"
townsend_trips = graph.edges.filter((col("src") == "Townsend at 7th") | (col("dst") == "Townsend at 7th"))

# Compter le nombre de trajets
townsend_trip_count = townsend_trips.count()

# Afficher le nombre de trajets
print(f"Nombre de trajets qui commencent ou se terminent à 'Townsend at 7th' : {townsend_trip_count}")



Nombre de trajets qui commencent ou se terminent à 'Townsend at 7th' : 9


7. Sommets n'étant jamais une destination d'un voyage qui commence à 'Spear at Folsom'

In [19]:
# Trouver les sommets qui n'ont jamais été destination d'un voyage partant de "Spear at Folsom"
spears_trips = graph.edges.filter(col("src") == "Spear at Folsom")
not_destinations = spears_trips.select("dst")

# Récupérer les sommets qui ne sont pas dans la liste des destinations
non_destination_vertices = graph.vertices.join(not_destinations, graph.vertices.id == not_destinations.dst, "left_anti")

# Afficher les résultats
non_destination_vertices.show()


+----------+--------------------+---------+-----------+---------+------------+------------+
|station_id|                  id|      lat|       long|dockcount|    landmark|installation|
+----------+--------------------+---------+-----------+---------+------------+------------+
|         2|San Jose Diridon ...|37.329732|-121.901782|       27|    San Jose|    8/6/2013|
|         3|San Jose Civic Ce...|37.330698|-121.888979|       15|    San Jose|    8/5/2013|
|         4|Santa Clara at Al...|37.333988|-121.894902|       11|    San Jose|    8/6/2013|
|         5|    Adobe on Almaden|37.331415|  -121.8932|       19|    San Jose|    8/5/2013|
|         6|    San Pedro Square|37.336721|-121.894074|       15|    San Jose|    8/7/2013|
|         7|Paseo de San Antonio|37.333798|-121.886943|       15|    San Jose|    8/7/2013|
|         8| San Salvador at 1st|37.330165|-121.885831|       15|    San Jose|    8/5/2013|
|         9|           Japantown|37.348742|-121.894715|       15|    San Jose|  

8. Station avec le nombre maximum de voyages entrants

In [20]:
# Compter les trajets entrants pour chaque station
incoming_trips = graph.edges.groupBy("dst").count()

# Trouver la station avec le plus grand nombre de trajets entrants
max_incoming_station = incoming_trips.orderBy(col("count").desc()).limit(1)

# Afficher la station avec le nombre maximum de trajets entrants
max_incoming_station.show()


+--------------------+-----+
|                 dst|count|
+--------------------+-----+
|San Francisco Cal...|    9|
+--------------------+-----+



9. Voyage avec la plus grande durée

In [13]:
from pyspark.sql.functions import max

# Calculer la durée maximale
max_duration = trip_edges.agg(max("Duration")).collect()[0][0]

# Filtrer pour obtenir le(s) trajet(s) ayant cette durée maximale
longest_trip = trip_edges.filter(trip_edges.Duration == max_duration)
longest_trip.show()





+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
|Trip ID|Duration|     Start Date|                 src|Start Terminal|       End Date|                 dst|End Terminal|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+
| 913386|    1808|8/31/2015 20:23|Embarcadero at Br...|            54|8/31/2015 20:53|Harry Bridges Pla...|          50|   524|     Subscriber|   94105|
+-------+--------+---------------+--------------------+--------------+---------------+--------------------+------------+------+---------------+--------+



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

10. Créer un sous-graphe pour les voyages se terminant ou commençant à ‘Townsend at 7th’

In [12]:
# Créer un sous-graphe pour les voyages qui commencent ou se terminent à 'Townsend at 7th'
subgraph = GraphFrame(
    station_vertices,
    graph.edges.filter((graph.edges.src == 'Townsend at 7th') | (graph.edges.dst == 'Townsend at 7th'))
)


11. Recherche de motifs : chemins en "triangle" entre trois stations

In [12]:
# Rechercher les triangles dans le graphe
triangles = graph.find("(a)-[ab]->(b); (b)-[bc]->(c); (c)-[ca]->(a)")
triangles.show()


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|                  ab|                   b|                  bc|                   c|                  ca|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|{60, Embarcadero ...|{913450, 255, 8/3...|{74, Steuart at M...|{913315, 518, 8/3...|{61, 2nd at Towns...|{913383, 1435, 8/...|
|{60, Embarcadero ...|{913443, 691, 8/3...|{77, Market at Sa...|{913403, 497, 8/3...|{61, 2nd at Towns...|{913383, 1435, 8/...|
|{69, San Francisc...|{913434, 283, 8/3...|{65, Townsend at ...|{913371, 663, 8/3...|{49, Spear at Fol...|{913432, 703, 8/3...|
|{49, Spear at Fol...|{913432, 703, 8/3...|{69, San Francisc...|{913308, 185, 8/3...|{61, 2nd at Towns...|{913297, 501, 8/3...|
|{49, Spear at Fol...|{913432, 703, 8/3...|{69, San Francisc...|{913316, 300, 8/3...|{65, Townsend at ..

12. Chemins de trois sommets passant par 'Townsend at 7th'

In [22]:
# Rechercher des chemins de trois sommets commençant par 'Townsend at 7th'
paths_through_townsend = graph.find("(a)-[ab]->(b); (b)-[bc]->(c)") \
    .filter("a.id = 'Townsend at 7th'")

# Afficher les résultats avec les identifiants de chaque sommet
paths_through_townsend.select("a.id", "b.id", "c.id").show()


+---------------+--------------------+--------------------+
|             id|                  id|                  id|
+---------------+--------------------+--------------------+
|Townsend at 7th|Harry Bridges Pla...|     2nd at Townsend|
|Townsend at 7th|Harry Bridges Pla...|Embarcadero at Br...|
|Townsend at 7th|Harry Bridges Pla...|San Francisco Cal...|
|Townsend at 7th|     Spear at Folsom|     2nd at Townsend|
|Townsend at 7th|     Spear at Folsom|    Davis at Jackson|
|Townsend at 7th|     Spear at Folsom|     Townsend at 7th|
|Townsend at 7th|     Spear at Folsom|     2nd at Townsend|
|Townsend at 7th|     Spear at Folsom|San Francisco Cal...|
+---------------+--------------------+--------------------+

